In [212]:
import pandas as pd
import random
import numpy as np
from pprint import pprint

## 1. Step of the Algorithm

In [213]:
def frame_split(df, test_size_in_percent):
    
    test_size = round(test_size_in_percent/100 * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
        
    return train_df, test_df

In [214]:
# load data
label_name = "type"
df = pd.read_csv("extracsvAll.csv")
df.drop(label_name,axis=1)
df = df.round(1)
df.head()

,approxAnzahl,areaRatio,circleLike,contureDistance,conturenAnzahl,farPoint,keyPointsAnzahl,mean_val,middlepointDistance,minimal_ratio,minimal_ratio2,momentpointDistance,rectangleLike,seradity,seradity2,solidity,solidity2,steiner,type
0,26,0.0,0.4,1.0,3,65806,434,254.0,-2.8,2.7,1.0,-8.2,1.0,0.6,1.0,0.7,1.0,5.1,Dosenoeffner
1,27,0.1,0.4,1.1,2,29660,432,240.2,-1.0,2.0,1.7,24.3,1.2,0.7,1.0,0.5,1.0,30.2,Korkenzieher
2,20,0.1,0.5,1.3,2,7145,418,228.5,7.3,3.0,1.6,23.8,1.5,0.9,1.0,0.7,1.0,16.3,Flaschenoeffner
3,20,0.0,0.3,1.5,3,27976,322,249.4,7.5,1.6,1.8,-0.0,0.3,0.8,1.0,0.7,1.0,17.7,Dosenoeffner
4,19,0.0,0.4,1.1,2,56244,407,253.0,4.5,2.3,1.2,-7.6,1.0,0.7,1.0,0.6,1.0,19.2,Dosenoeffner


In [215]:
df_train, df_test = frame_split(df, test_size_in_percent=20)
test_labels = df_test[label_name]
df_test.drop(label_name,axis=1)

,approxAnzahl,areaRatio,circleLike,contureDistance,conturenAnzahl,farPoint,keyPointsAnzahl,mean_val,middlepointDistance,minimal_ratio,minimal_ratio2,momentpointDistance,rectangleLike,seradity,seradity2,solidity,solidity2,steiner
33,18,0.1,0.3,0.9,2,8366,284,236.6,22.4,2.9,1.6,65.0,0.7,0.9,1.0,0.9,1.0,4.1
149,11,0.1,0.1,1.0,2,551,326,228.1,42.3,2.2,1.5,88.4,0.1,1.0,0.9,1.0,1.0,5.0
628,20,0.1,0.4,1.0,3,10107,391,227.2,10.3,2.7,1.6,38.0,1.0,0.9,0.9,0.8,0.9,21.2
269,20,1.0,0.3,1.1,1,20657,427,255.0,11.7,2.3,2.3,34.4,0.7,0.8,0.8,0.8,0.8,25.7
26,21,1.0,0.2,1.4,1,6330,313,255.0,6.3,2.2,2.2,29.1,0.6,0.8,0.8,0.8,0.8,21.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,23,0.0,0.4,1.4,2,22703,424,249.8,10.5,1.5,1.3,13.0,0.9,0.8,1.0,0.6,1.0,37.6
312,27,0.1,0.4,1.3,4,37456,438,213.9,23.2,1.2,4.0,30.6,0.8,0.6,0.9,0.5,0.9,23.0
223,30,0.0,0.2,1.4,2,22193,449,251.8,-6.3,1.3,1.0,54.5,1.4,0.8,1.0,0.5,1.0,65.0
416,28,0.1,0.3,1.1,5,28104,448,224.2,26.9,1.9,2.1,53.0,0.6,0.7,0.9,0.7,0.9,6.1


In [216]:
def wahrscheinlichkeiten(df, label_column):
    table = {}

    # determine values for the label
    value_counts = df[label_column].value_counts().sort_index()
    table["class_names"] = value_counts.index.to_numpy()
    table["class_counts"] = value_counts.values

    # determine probabilities for the features
    for feature in df.drop(label_column, axis=1).columns:
        table[feature] = {}

        # determine counts
        counts = df.groupby(label_column)[feature].value_counts()
        df_counts = counts.unstack(label_column)

        # add one count to avoid "problem of rare values"
        if df_counts.isna().any(axis=None):
            df_counts.fillna(value=0, inplace=True)
            df_counts += 1

        # calculate probabilities
        df_probabilities = df_counts / df_counts.sum()
        for value in df_probabilities.index:
            probabilities = df_probabilities.loc[value].to_numpy()
            table[feature][value] = probabilities
            
    return table

In [217]:
bayes_table = wahrscheinlichkeiten(df_train, label_column=label_name)
pprint(bayes_table)

{'approxAnzahl': {4: array([0.00843882, 0.09787234, 0.04366812]),
                  6: array([0.00421941, 0.01702128, 0.00436681]),
                  8: array([0.00421941, 0.01702128, 0.00436681]),
                  9: array([0.00421941, 0.00851064, 0.00436681]),
                  11: array([0.01687764, 0.03404255, 0.00436681]),
                  12: array([0.02531646, 0.07659574, 0.00436681]),
                  13: array([0.02109705, 0.02978723, 0.00873362]),
                  14: array([0.00421941, 0.05957447, 0.00873362]),
                  15: array([0.04219409, 0.04255319, 0.02183406]),
                  16: array([0.03375527, 0.05957447, 0.01310044]),
                  17: array([0.05907173, 0.05957447, 0.00873362]),
                  18: array([0.08860759, 0.07234043, 0.01746725]),
                  19: array([0.06751055, 0.02978723, 0.02183406]),
                  20: array([0.10970464, 0.02978723, 0.05240175]),
                  21: array([0.08016878, 0.05106383, 0.0349345 ]),

## 2. Step of the Algorithm

In [218]:
def example(row,bayes_table):
    
    class_estimates = bayes_table["class_counts"]
    for feature in row.index:

        try:
            value = row[feature]
            probabilities = bayes_table[feature][value]
            class_estimates = class_estimates * probabilities

        # skip in case "value" only occurs in test set but not in train set
        # (i.e. "value" is not in "lookup_table")
        except KeyError:
            continue

    index_max_class = class_estimates.argmax()
    prediction = bayes_table["class_names"][index_max_class]
    
    return prediction

In [219]:
def calculate_keynumbers_bayes(df, bayes_table):

    df["classification"] = df.apply(example, axis=1, args=(bayes_table,))
    df["classification_correct"] = df["classification"] == df[label_name]
    overall_accuracy = df["classification_correct"].mean()

    label_column = df[label_name]
    unique_classes, _ = np.unique(label_column, return_counts=True)
        
    keynumbers = []

    for uc in unique_classes:
            
        tp_df = df.loc[(df[label_name] == uc) & (df["classification_correct"] == True)]
        fp_df = df.loc[(df[label_name] == uc) & (df["classification_correct"] == False)]
        tn_df = df.loc[(df[label_name] != uc) & (df["classification_correct"] == True)]
        fn_df = df.loc[(df[label_name] != uc) & (df["classification_correct"] == False)]

        true_positiv, _ = tp_df.shape
        false_positiv, _ = fp_df.shape
        true_negative, _ = tn_df.shape
        false_negative, _ = fn_df.shape

        recall = true_positiv / (true_positiv+false_negative)
        precision = true_positiv / (true_positiv+false_positiv)
        f1score = 2 * (precision * recall) / (precision + recall)

        keynumbers.append({"Label":uc, "recall":recall, "precision":precision, "f1score":f1score})
        
        
    return overall_accuracy, keynumbers

# Check Accuracy

In [220]:
accuracy, keynumbers = calculate_keynumbers_bayes(df_test, bayes_table)
accuracy

0.8466666666666667

In [221]:
keynumbers = pd.DataFrame(keynumbers)
keynumbers

,Label,recall,precision,f1score
0,Dosenoeffner,0.754098,0.851852,0.800000
1,Flaschenoeffner,0.698113,0.840909,0.762887
2,Korkenzieher,0.745763,0.846154,0.792793
